In [11]:
import numpy as np 
import pandas as pd 
import matplotlib as plt 
import seaborn as sns 
import tensorflow as tf
from tensorflow import keras
import tensorflow.python.platform.build_info as build
from database_connection import DBConn
from sqlalchemy import text 

In [2]:
# check gpu availability 
print(f"TF is cuda build: {build.build_info['is_cuda_build']}")
print(f"Device list: {tf.config.list_physical_devices('GPU')}")

TF is cuda build: True
Device list: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [12]:
# setup database connection 
HOST = 'localhost'
USER = 'db_user'
PORT = 5438
DATABASE = 'shot_db'
PASSWORD = 'LetMeIn'
TABLE_NAME = 'shot_data_table'

db = DBConn()

In [14]:
print(db.execute_query())

None
